# Assignment 1: Credit Card Statement Data Extraction

**Objective:** Extract key information from credit card statement images using OpenAI Vision API and LangChain

**Data to Extract:**
- Customer Name
- Card Account Number
- Statement Date
- Total Amount Due
- Minimum Amount Due
- Due Date

## Step 1: Import Required Libraries

In [1]:
import os
import base64
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

## Step 2: Load Environment Variables and Initialize Model

In [2]:
# Load environment variables
load_dotenv()

# Initialize OpenAI model with vision capabilities
llm = ChatOpenAI(
    model="gpt-4o-mini",  # GPT-4o-mini has vision capabilities
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

print("Model initialized successfully!")

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Step 3: Define Data Structure for Extraction

In [ ]:
# Define the structure for extracted data using Pydantic
class CreditCardStatement(BaseModel):
    customer_name: str = Field(description="Full name of the credit card holder")
    card_account_number: str = Field(description="Credit card account number (masked if needed)")
    statement_date: str = Field(description="Statement date in DD/MM/YYYY format")
    total_amount_due: str = Field(description="Total amount due including currency symbol")
    minimum_amount_due: str = Field(description="Minimum amount due including currency symbol")
    due_date: str = Field(description="Payment due date in DD/MM/YYYY format")

# Create parser
parser = JsonOutputParser(pydantic_object=CreditCardStatement)

print("Data structure defined!")
print("\nExpected fields:")
for field_name in CreditCardStatement.model_fields.keys():
    print(f"  - {field_name}")

Data structure defined!

Expected fields:
  - customer_name
  - card_account_number
  - statement_date
  - total_amount_due
  - minimum_amount_due
  - due_date


## Step 4: Load and Encode Image

In [ ]:
# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Image path - Change this to your statement image
image_path = "sample_statements/statement1.png"

# Encode the image
base64_image = encode_image(image_path)

print(f"Image loaded: {image_path}")
print(f"Image encoded successfully! (Size: {len(base64_image)} characters)")

Image loaded: sample_statements/statement1.png
Image encoded successfully! (Size: 233300 characters)


## Step 5: Create Prompt for AI

In [ ]:
# Create the prompt
prompt = f"""You are a credit card statement data extraction expert.

Analyze this credit card statement image and extract the following information:
- Customer Name
- Card Account Number
- Statement Date
- Total Amount Due
- Minimum Amount Due
- Due Date

Return the data in the following JSON format:
{parser.get_format_instructions()}

Be precise and extract exactly what you see in the image.
"""

print("Prompt created successfully!")

Prompt created successfully!


## Step 6: Send Image to AI and Extract Data

In [ ]:
# Create message with image
message = HumanMessage(
    content=[
        {"type": "text", "text": prompt},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/png;base64,{base64_image}"}
        }
    ]
)

# Get response from AI
print("[INFO] Analyzing credit card statement image...\n")
response = llm.invoke([message])

# Parse the response
extracted_data = parser.parse(response.content)

print("[SUCCESS] Data extracted successfully!")

[INFO] Analyzing credit card statement image...

[SUCCESS] Data extracted successfully!


## Step 7: Display Extracted Data

In [ ]:
# Display results in a formatted way
print("="*60)
print("EXTRACTED CREDIT CARD STATEMENT DATA")
print("="*60)
print(f"\nCustomer Name:        {extracted_data['customer_name']}")
print(f"Card Account Number:  {extracted_data['card_account_number']}")
print(f"Statement Date:       {extracted_data['statement_date']}")
print(f"Total Amount Due:     {extracted_data['total_amount_due']}")
print(f"Minimum Amount Due:   {extracted_data['minimum_amount_due']}")
print(f"Due Date:             {extracted_data['due_date']}")
print("\n" + "="*60)
print("[SUCCESS] Data extraction completed!")
print("="*60)

EXTRACTED CREDIT CARD STATEMENT DATA

Customer Name:        MR. HITESH KUMAR
Card Account Number:  4375 XXXX XXXX 8007
Statement Date:       23/04/2018
Total Amount Due:     ₹ 8,795.59
Minimum Amount Due:   ₹ 6,620.00
Due Date:             11/05/2018

[SUCCESS] Data extraction completed!


## Optional: View Raw JSON Output

In [ ]:
# Display the raw JSON data
import json

print("Raw JSON Output:")
print(json.dumps(extracted_data, indent=2))

Raw JSON Output:
{
  "customer_name": "MR. HITESH KUMAR",
  "card_account_number": "4375 XXXX XXXX 8007",
  "statement_date": "23/04/2018",
  "total_amount_due": "\u20b9 8,795.59",
  "minimum_amount_due": "\u20b9 6,620.00",
  "due_date": "11/05/2018"
}
